In [3]:
import tensorflow as tf
from tensorflow import keras
from scipy import stats
import numpy as np
import glob
import pandas as pd
import matplotlib.pyplot as plt

from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras import layers

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

%matplotlib inline

In [4]:
import glob
import pandas as pd

In [5]:
link = "https://s3-us-west-1.amazonaws.com/helen.assignment/sentiment_master.csv"
df = pd.read_csv(link, names=["comments", "sentiment", "magnitude"])

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (2,3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
df.drop(['magnitude'], axis=1, inplace=True)

In [9]:
df['sentiment'] = pd.to_numeric(df['sentiment'],errors='coerce')
df['sentiment'] = df['sentiment'].apply(lambda x: 1 if x > 0 else 0)
# df.drop(index='NaN', axis=0)

In [10]:
df

,comments,sentiment
NaN,comments,0
0.0,Hi\nThis backpack which is Show in“read the St...,1
1.0,I place an order and looking forward to using ...,1
2.0,Waiting for an year for such an amazing produc...,1
3.0,Ordered! The science of effectively having an ...,1
...,...,...
218313.0,Dear Joao and all the Backers!,0
218314.0,I will ship all the Kal pieces on this Friday....,1
218315.0,Can you post an update? And give me some feedb...,0
218316.0,Thanks Andrew for everything! You will have a ...,1


In [11]:
x = df['comments'].values[:80000]
y = df['sentiment'].values[:80000]

In [12]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=123)

In [27]:
x_train

array(['@Avinash Magdum: \nok，We have added.',
       'To set the IP address manually, click advanced on the first page (the one that you select the Wi-Fi networks). All of the IP address fields are there.  ',
       'Order ID\n278\nstill waiting', ...,
       'Looking great guys!!  Keep up the good work.',
       'Dave. When will you shipping?',
       '... Not too far from where I live.'], dtype=object)

In [13]:
# num_words = number of keywords 
tokenizer = Tokenizer(num_words=100)
tokenizer.fit_on_texts(x)
xtrain= tokenizer.texts_to_sequences(x_train)
xtest= tokenizer.texts_to_sequences(x_test) 

In [14]:
# maxlen = maximum length per comment to look at
maxlen=20
xtrain=pad_sequences(xtrain,padding='post', maxlen=maxlen)
xtest=pad_sequences(xtest,padding='post', maxlen=maxlen)

In [15]:
xtrain

array([[13,  0,  0, ...,  0,  0,  0],
       [ 2,  1, 15, ...,  0,  0,  0],
       [60, 55,  0, ...,  0,  0,  0],
       ...,
       [72, 68,  1, ...,  0,  0,  0],
       [40, 21,  6, ...,  0,  0,  0],
       [22, 41,  3, ...,  0,  0,  0]], dtype=int32)

In [16]:
print(x_train[4])
print(xtrain[4])


Will there be programs and apps available to make uploading data easy?
[21 52 18  4  2  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]


In [17]:
vocab_size=len(tokenizer.word_index)+1
vocab_size

52576

In [18]:
embedding_dim=50
model=Sequential()
model.add(layers.Embedding(input_dim=vocab_size,
         output_dim=embedding_dim,
         input_length=maxlen))
model.add(layers.LSTM(units=50,return_sequences=True))
model.add(layers.LSTM(units=10))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(8))
model.add(layers.Dense(1, activation="sigmoid"))
model.compile(optimizer="adam", loss="binary_crossentropy", 
     metrics=['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 50)            2628800   
_________________________________________________________________
lstm (LSTM)                  (None, 20, 50)            20200     
_________________________________________________________________
lstm_1 (LSTM)                (None, 10)                2440      
_________________________________________________________________
dropout (Dropout)            (None, 10)                0         
_________________________________________________________________
dense (Dense)                (None, 8)                 88        
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 9         
Total params: 2,651,537
Trainable params: 2,651,537
Non-trainable params: 0
______________________________________________

In [28]:
model.fit(xtrain,y_train, epochs=20, batch_size=16, verbose=True)

Train on 72000 samples
Epoch 1/20
72000/72000 [==============================] - 351s 5ms/sample - loss: 0.5529 - accuracy: 0.7141
Epoch 2/20
72000/72000 [==============================] - 356s 5ms/sample - loss: 0.5344 - accuracy: 0.7276
Epoch 3/20
72000/72000 [==============================] - 373s 5ms/sample - loss: 0.5293 - accuracy: 0.7316
Epoch 4/20
72000/72000 [==============================] - 342s 5ms/sample - loss: 0.5256 - accuracy: 0.7347
Epoch 5/20
72000/72000 [==============================] - 472s 7ms/sample - loss: 0.5212 - accuracy: 0.7350
Epoch 6/20
72000/72000 [==============================] - 424s 6ms/sample - loss: 0.5187 - accuracy: 0.7397
Epoch 7/20
72000/72000 [==============================] - 381s 5ms/sample - loss: 0.5144 - accuracy: 0.7425
Epoch 8/20
72000/72000 [==============================] - 378s 5ms/sample - loss: 0.5100 - accuracy: 0.7449
Epoch 9/20
72000/72000 [==============================] - 418s 6ms/sample - loss: 0.5057 - accuracy: 0.7502
Epoch

In [30]:
model.save('my_half_model')

INFO:tensorflow:Assets written to: my_half_model/assets


In [22]:
my_model = keras.models.load_model('my_half_model')

In [24]:
loss, acc = my_model.evaluate(xtrain, y_train, verbose=True)
print("Training Accuracy: ", acc.round(2))

 
loss, acc = my_model.evaluate(xtest, y_test, verbose=True)
print("Test Accuracy: ", acc.round(2))


72000/72000 [==============================] - 36s 502us/sample - loss: 0.4130 - accuracy: 0.8021
Training Accuracy:  0.8
8000/8000 [==============================] - 3s 329us/sample - loss: 0.5338 - accuracy: 0.7369
Test Accuracy:  0.74


## Predicting Test Data


In [25]:
ypred=my_model.predict(xtest)

In [26]:
result=zip(x_test[:10], y_test[:10], ypred[10])
for i in result:
 print(i)

("To be honest, I don't have a problem with the water flow. I don't chug my water so I don't suck more than I need to. I think you all are over reacting because you have waited a while so you are expecting an absolutely perfect product because of the wait. I don't have any problems with it and love it in every way.", 1, 0.00235788)


In [29]:
ypred

array([[0.45604467],
       [0.11726645],
       [0.7339141 ],
       ...,
       [0.53813964],
       [0.00257869],
       [0.5505388 ]], dtype=float32)